<a href="https://colab.research.google.com/github/ValenChamizo/Observatorio_Vial/blob/main/Observatorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 📦 Instalación de librerías necesarias
!pip install dash jupyter-dash pandas plotly sqlalchemy --quiet  # Removed pymysql

# 📚 Importaciones
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, State
from sqlalchemy import create_engine
from datetime import datetime
import numpy as np

# 🔗 Conexión a SQLite
# The database file will be created in the same directory as your script/notebook.
engine = create_engine("sqlite:///mydb.db")  # Changed connection string

# 🧱 Cargar datos desde la base de datos
def cargar_datos():
    df = pd.read_sql("SELECT * FROM Incidente_Trafico", engine)
    df["fecha"] = pd.to_datetime(df["fecha"])
    return df
df_incidentes = cargar_datos()

# 🎯 Filtros iniciales
anios = sorted(df_incidentes["fecha"].dt.year.unique())
ubicaciones = sorted(df_incidentes["ubicacion"].unique())

# 🖼️ Layout de la app
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Observatorio Vial de Manizales", style={"textAlign": "center", "color": "#003366"}),

    # 🔍 Filtros
    html.Div([
        html.Label("Año:", style={"color": "#003366"}),
        dcc.Dropdown(
            id="filtro-anio",
            options=[{"label": str(anio), "value": anio} for anio in anios],
            multi=False,
            style={"width": "200px"}
        ),
        html.Label("Ubicación:", style={"color": "#003366", "marginLeft": "20px"}),
        dcc.Dropdown(
            id="filtro-ubicacion",
            options=[{"label": ubicacion, "value": ubicacion} for ubicacion in ubicaciones],
            multi=False,
            style={"width": "200px"}
        ),
    ], style={"display": "flex", "alignItems": "center", "justifyContent": "center", "padding": "20px"}),

    # 🔢 KPIs
    html.Div([
        html.Div([
            html.H3("Total Incidentes", style={"color": "#003366"}),
            html.P(id="total-incidentes", style={"fontSize": "24px", "fontWeight": "bold"})
        ], style={"padding": "10px", "border": "1px solid #ccc", "borderRadius": "5px", "margin": "10px"}),
        html.Div([
            html.H3("Total Accidentes", style={"color": "#003366"}),
            html.P(id="total-accidentes", style={"fontSize": "24px", "fontWeight": "bold"})
        ], style={"padding": "10px", "border": "1px solid #ccc", "borderRadius": "5px", "margin": "10px"}),
        html.Div([
            html.H3("Total Congestiones", style={"color": "#003366"}),
            html.P(id="total-congestiones", style={"fontSize": "24px", "fontWeight": "bold"})
        ], style={"padding": "10px", "border": "1px solid #ccc", "borderRadius": "5px", "margin": "10px"}),
    ], style={"display": "flex", "justifyContent": "space-around", "flexWrap": "wrap"}),

    # 📊 Gráficos
    html.Div([
        dcc.Graph(id="bar-chart-mes"),
        dcc.Graph(id="pie-chart-tipo")
    ], style={"display": "flex", "justifyContent": "space-around", "flexWrap": "wrap"}),

    # 🗺️ Mapa de incidentes
    html.Div([
        html.H2("Mapa de Incidentes", style={"color": "#003366"}),
        dcc.Graph(id="mapa-incidentes")
    ], style={"padding": "20px"}),

    # 📋 Tabla de últimos incidentes
    html.Div([
        html.H2("Últimos Incidentes", style={"color": "#003366"}),
        html.Table(id="tabla-ultimos-incidentes", style={"width": "100%", "borderCollapse": "collapse"})
    ], style={"padding": "20px"}),

    # 📝 Formulario para añadir nuevos incidentes
    html.Div([
        html.H2("Añadir Nuevo Incidente", style={"color": "#003366"}),
        html.Div([
            html.Label("Ubicación:", style={"color": "#003366"}),
            dcc.Input(id="input-ubicacion", type="text", placeholder="Ingrese ubicación", style={"marginRight": "10px"}),
            html.Label("Tipo Incidente:", style={"color": "#003366"}),
            dcc.Input(id="input-tipo-incidente", type="text", placeholder="Ingrese tipo", style={"marginRight": "10px"}),
            html.Label("Fecha (DD/MM/AAAA):", style={"color": "#003366"}),
            dcc.Input(id="input-fecha", type="text", placeholder="DD/MM/AAAA", style={"marginRight": "10px"}),
            html.Label("Estado:", style={"color": "#003366"}),
            dcc.Input(id="input-estado", type="text", placeholder="Ingrese estado", style={"marginRight": "10px"}),
            html.Label("Cantidad Vehículos:", style={"color": "#003366"}),
            dcc.Input(id="input-cantidad-vehiculos", type="number", placeholder="Ingrese cantidad", style={"marginRight": "10px"}),
            html.Button("Agregar", id="boton-agregar", n_clicks=0, style={"backgroundColor": "#4CAF50", "color": "white", "padding": "5px 10px", "border": "none", "borderRadius": "5px", "cursor": "pointer"})
        ], style={"display": "flex", "flexWrap": "wrap", "alignItems": "center"})
    ], style={"padding": "20px"})
])

# 🔁 Callback para actualizar dashboard y enviar nuevos incidentes a SQLite
@app.callback(
    Output("bar-chart-mes", "figure"),
    Output("pie-chart-tipo", "figure"),
    Output("mapa-incidentes", "figure"),
    Output("total-incidentes", "children"),
    Output("total-accidentes", "children"),
    Output("total-congestiones", "children"),
    Output("tabla-ultimos-incidentes", "children"),
    Input("filtro-anio", "value"),
    Input("filtro-ubicacion", "value"),
    Input("boton-agregar", "n_clicks"),
    State("input-ubicacion", "value"),
    State("input-tipo-incidente", "value"),
    State("input-fecha", "value"),
    State("input-estado", "value"),
    State("input-cantidad-vehiculos", "value")
)
def actualizar_dashboard(anio, ubicacion, n_clicks, ubicacion_nueva, tipo_nuevo, fecha_nueva, estado_nuevo, cantidad_nueva):
    if n_clicks > 0 and all([ubicacion_nueva, tipo_nuevo, fecha_nueva, estado_nuevo, cantidad_nueva]):
        try:
            fecha_obj = datetime.strptime(fecha_nueva, "%d/%m/%Y")
            nuevo = pd.DataFrame([{
                "ubicacion": ubicacion_nueva,
                "tipo_incidente": tipo_nuevo,
                "fecha": fecha_obj,
                "estado": estado_nuevo,
                "cantidad_vehiculos": cantidad_nueva,
                "latitud": np.random.uniform(2.4, 2.5),
                "longitud": np.random.uniform(-76.6, -76.5)
            }])
            nuevo['fecha'] = pd.to_datetime(nuevo['fecha']) # Ensure datetime object
            nuevo.to_sql("Incidente_Trafico", engine, if_exists="append", index=False)
        except Exception as e:
            print(f"Error al insertar: {e}")

    df = cargar_datos()
    if anio:
        df = df[df["fecha"].dt.year == anio]
    if ubicacion:
        df = df[df["ubicacion"] == ubicacion]

    # KPIs
    total_incidentes = len(df)
    total_accidentes = len(df[df["tipo_incidente"] == "Accidente"])
    total_congestiones = len(df[df["tipo_incidente"] == "Congestión"])

    # Barras por mes
    df["mes"] = df["fecha"].dt.strftime("%B")
    df["mes_num"] = df["fecha"].dt.month
    df_mes = df.groupby("mes").size().reset_index(name="Numero de Incidentes")
    df_mes["mes_num"] = df_mes["mes"].apply(lambda x: datetime.strptime(x, "%B").month)
    df_mes = df_mes.sort_values("mes_num")
    bar_fig = px.bar(df_mes, x="mes", y="Numero de Incidentes", title="Incidentes por Mes", template="plotly_dark")

    # Pie chart por tipo
    tipo_data = df["tipo_incidente"].value_counts().reset_index()
    tipo_data.columns = ["tipo_incidente", "count"]
    pie_fig = px.pie(tipo_data, names="tipo_incidente", values="count", title="Distribución por Tipo", template="plotly_dark")

    # Mapa
    if "latitud" in df.columns and "longitud" in df.columns:
        fig_mapa = px.scatter_mapbox(
            df,
            lat="latitud",
            lon="longitud",
            color="tipo_incidente",
            hover_name="ubicacion",
            zoom=12,
            title="Mapa de Incidentes"
        )
        fig_mapa.update_layout(mapbox_style="open-street-map", margin={"r":0,"t":40,"l":0,"b":0})
    else:
        fig_mapa = {}

    # Últimos incidentes
    ultimos = df.sort_values(by="fecha", ascending=False).head(5)
    ultimos["fecha"] = ultimos["fecha"].dt.strftime("%d/%m/%Y")
    columnas = ["ubicacion", "tipo_incidente", "fecha", "estado", "cantidad_vehiculos"]
    tabla = [html.Tr([html.Th(col.capitalize().replace("_", " ")) for col in columnas])] + \
            [html.Tr([html.Td(ultimos.iloc[i][col]) for col in columnas]) for i in range(len(ultimos))]

    return bar_fig, pie_fig, fig_mapa, total_incidentes, total_accidentes, total_congestiones, tabla

# 🚀 Ejecutar en línea (Google Colab)
if __name__ == '__main__':
    app.run(mode="inline")

/usr/local/lib/python3.11/dist-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

# Nueva sección